<a href="https://colab.research.google.com/github/softconrob/Master-Thesis/blob/main/Augmenting_Datasets_with_Random_Noise_for_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports and Data


In [ ]:
# install libraries
!pip install scikit-learn pandas numpy
!pip install torch==1.12.1 sentence-transformers==2.2.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 52.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 57.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 59.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 62.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatib

In [ ]:
# imports
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import paired_cosine_distances
import torch
import os
import torch.nn.functional as F

In [ ]:
# connect to drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import xml.etree.ElementTree as ET
import os

# Path to folder
path_to_datasets = "/content/drive/My Drive/UZH/MA/Datasets/"

# Function to parse TMX files and convert them into DataFrames
def parse_tmx(file_path):
    if not os.path.isfile(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")

    tree = ET.parse(file_path)
    root = tree.getroot()
    namespaces = {'xml': 'http://www.w3.org/XML/1998/namespace'}

    translations = []

    for tu in root.findall('.//tu'):
        de_text = None
        fr_text = None

        for tuv in tu.findall('tuv'):
            lang = tuv.attrib['{http://www.w3.org/XML/1998/namespace}lang']
            seg = tuv.find('seg').text
            if lang == 'de':
                de_text = seg
            elif lang == 'fr':
                fr_text = seg

        if de_text and fr_text:
            translations.append({'de': de_text, 'fr': fr_text})

    return pd.DataFrame(translations)

# Full paths to TMX files
mono_file_path = os.path.join(path_to_datasets, 'NeuLab-TedTalks_de-fr', 'de-fr.tmx')
cross_file_path = os.path.join(path_to_datasets, 'ELRC-CORDIS_News_de-fr', 'de-fr.tmx')

# Load datasets
mono_df = parse_tmx(mono_file_path)
cross_df = parse_tmx(cross_file_path)

# Display the dataframes
print(mono_df.head())
print(cross_df.head())


                                                  de  \
0  Unter den schwierigen Problemen, mit denen wir...   
1  Wir Diplomaten sind dazu ausgebildet worden, m...   
2              Und damit sind wir voll ausgelastet:    
3  Es gibt Handel, Entwaffnung und grenzüberschre...   
4  Doch das Bild ändert sich. Neue Hauptakteure t...   

                                                  fr  
0  Parmi tous les déficits inquiétants contre les...  
1  Nous qui sommes des diplomates, sommes formés ...  
2  Et je peux vous dire, que notre agenda est ple...  
3  Il y a le commerce, il y a le désarmement, il ...  
4  Mais la situation évolue, et nous constatons q...  
                                                  de  \
0                       Eine bahnbrechende Leistung    
1  Ein Beispiel dafür ist die Entwicklung von neu...   
2  "Eine wesentliche Motivation für diese Arbeit ...   
3  Gleichzeitig muss der Feuerwehrmann volles Ver...   
4  Diese begann im Mai 2015 und wird einen Schwer... 

# Introduce Random Noise to Parallel French-German Dataset

In [ ]:
import pandas as pd
import random

# Function to introduce random noise
def introduce_ocr_like_noise(text, noise_rate=0.05):
    characters = list(text)
    n = len(characters)
    num_changes = int(n * noise_rate)
    for _ in range(num_changes):
        operation = random.choice(["delete", "insert", "substitute"])
        index = random.randint(0, n - 1)

        if operation == "delete":
            characters.pop(index)
        elif operation == "insert":
            characters.insert(index, random.choice("abcdefghijklmnopqrstuvwxyz"))
        elif operation == "substitute":
            characters[index] = random.choice("abcdefghijklmnopqrstuvwxyz")

        # Adjusting length for the next iteration
        n = len(characters)

    return ''.join(characters)

# Function to add noisy columns to the dataframe
def add_noisy_columns(df, lang, noise_levels):
    for noise in noise_levels:
        new_column_name = f"{lang}_{str(noise).replace('.', '')}"
        df[new_column_name] = df[lang].apply(introduce_ocr_like_noise, noise_rate=noise)
    return df

# Define noise levels
noise_levels = [0.05, 0.1, 0.15]

# Add noisy columns to mono_df
mono_df = add_noisy_columns(mono_df, 'de', noise_levels)
mono_df = add_noisy_columns(mono_df, 'fr', noise_levels)

# Add noisy columns to cross_df
cross_df = add_noisy_columns(cross_df, 'de', noise_levels)
cross_df = add_noisy_columns(cross_df, 'fr', noise_levels)

# Display the modified dataframes
print(mono_df.head())
print(cross_df.head())


                                                  de  \
0  Unter den schwierigen Problemen, mit denen wir...   
1  Wir Diplomaten sind dazu ausgebildet worden, m...   
2              Und damit sind wir voll ausgelastet:    
3  Es gibt Handel, Entwaffnung und grenzüberschre...   
4  Doch das Bild ändert sich. Neue Hauptakteure t...   

                                                  fr  \
0  Parmi tous les déficits inquiétants contre les...   
1  Nous qui sommes des diplomates, sommes formés ...   
2  Et je peux vous dire, que notre agenda est ple...   
3  Il y a le commerce, il y a le désarmement, il ...   
4  Mais la situation évolue, et nous constatons q...   

                                              de_005  \
0  Unteor den schwierigen Problemen, mit denen wi...   
1  Wir bDipblomaten sind dazu ausgebijdet wordsn,...   
2               Un damit sind wir voll ausgelastet:    
3  Es gibt Handel, Entwaffnuhng und grenzüberschr...   
4  Douh das Bild ändert sich. Neue Hauptakeure

In [ ]:
# Set the maximum number of columns displayed
pd.set_option('display.max_columns', None)  # None = unlimited

#display all column
display(mono_df.head(10))

,de,fr,de_005,de_01,de_015,fr_005,fr_01,fr_015
0,"Unter den schwierigen Problemen, mit denen wir...",Parmi tous les déficits inquiétants contre les...,"Unteor den schwierigen Problemen, mit denen wi...","Ugntler den bchwieriuen Probzlemen, mit denen ...","Untr dengschwierigen Problepen, mit deney wir ...",Parmi tous les déficits inquiétants contre leq...,Parmi jous les léficitms inaquiétants contb le...,Pami tous cles déficits inquitantsscontre lesv...
1,"Wir Diplomaten sind dazu ausgebildet worden, m...","Nous qui sommes des diplomates, sommes formés ...","Wir bDipblomaten sind dazu ausgebijdet wordsn,...","mhir Diplomaten sind dazu ausgebildet wordn,mi...","dir Dilomaten sina dazu aubgebilgdez worden, m...","xous qui sommes des diplomates, sommes formés ...","nous qui sommes desl diplomates,ssommes tfohfm...","Noqs quwi ommes des diplomates, zsjommes forms..."
2,Und damit sind wir voll ausgelastet:,"Et je peux vous dire, que notre agenda est ple...",Un damit sind wir voll ausgelastet:,Und damitosind wir voll ausugmelastet:,Und daqmit qand wir voll austelastet:s,"Et jepyeux vous dire, que notre agenda est ple...","Eto je pdux vous dire, ques notre agendaq est ...","Et je peux ous ire,b que notre ena est lein."
3,"Es gibt Handel, Entwaffnung und grenzüberschre...","Il y a le commerce, il y a le désarmement, il ...","Es gibt Handel, Entwaffnuhng und grenzüberschr...","Es gibt Handel, Etwaffnung uund grenüberscreit...","Ess gibt Handel,deEntwaffnung undagrendübhrmis...","Ilx y a le commerce, il y a le désarmemeynt, i...","In yh a le commercle, il yp a le désazmement, ...","Ilysb a lecgommerce, ail y a le désarmemeet, i..."
4,Doch das Bild ändert sich. Neue Hauptakteure t...,"Mais la situation évolue, et nous constatons q...",Douh das Bild ändert sich. Neue Hauptakeure ta...,Doch das ild ändert siich. peuet Hauptakteure ...,och das Bildakändeht sch. Neuq Hauptakgeube ta...,"Mais lg siuation évolu, et qous constatons qu'...","Mais la fstuation évolue, et nous bconstatonsr...","Mais la situatinw évolue, et nusconstatons qu'..."
5,"Diese bezeichnen wir lose als """" Gruppen """". S...",Nous les appelons vaguement « groupes ». Ils p...,"Diese beuzeichnen wir lose als """" Gruppen """". ...","Dies bezeichnreni wir nlose als """" Gruppen """"....","iese ezeichnenf wirlosse als ""s"" Gruppen "". Si...",Nous les appelons vaguemvnt « groupes ». Ils p...,Nous leappelons vagfuement « groupes ». Ils xp...,Nouis lews appeluonso vaguement « groupeso » I...
6,"Und wir wissen nicht, wie mit ihnen umzugehen ...",Et nous luttons sur la façon de traiter avec e...,"Und wcir wissen nichtw, wie mit ihnen umzugehe...","Und wir wisten nict, wi mit ihvnen uozugehen i...",Ubd wir wifssen nicgv wie mit ihnen umzufgrhen...,Et nous luttons skur la façon de traiter aveco...,Ext nocus luttose sur la façon de traiter avec...,Et nouw luyttons slrdlafaçonde traiter avec eux.
7,Die Einsatzregeln: wie und wann man sprechen s...,"Les règles d'engagement : comment dialoguer, q...",Die Einsatzregeeln: wie ndnwann man sprechen s...,DieaEinsatzregeln: wie und wann ma srechcen so...,Die Einosatzregeln:pwme und zann man prechengs...,"Leshrègles d'engagement : comment dialoguer, q...","Les règles d'engagemejnj : comment dialogpuer,...","Les ègles d'engragement : commenti dtaloger, q..."
8,"Ich zeige Ihnen hier eine Folie, die Ich zeige...",Permettez-moi de vous montrer une diapositive ...,"Ih zege Ihnen hiea eine Folie, die Ich zeice I...","Icl zeige Ibhnen hier eine Fole, die Ich zeige...","Ic zeige Ihnen hier epnex Folie, die Icx zzqig...",Permefttezd-omoi de vous monter une diapositiv...,ermettez-moi dse volus montrer une dipositive ...,Permeyttez-moi de vnous mortrer une iapositide...
9,bis heute darstellt. Das Grün bezeichnet tradi...,Vous voyez en vert c'est un conflit interétati...,bis heute darstelt. Das Grün bezeichnet tradit...,bi nteute darstellt. Ds Grün bezeichnet tradpi...,bish lheutedarstqellt. as rün bezechnetc tradi...,Vous voyez en vert c'est un onflit in

In [ ]:
# Set the maximum number of columns displayed
pd.set_option('display.max_columns', None)  # None = unlimited

#display all column
display(cross_df.head(10))

,de,fr,de_005,de_01,de_015,fr_005,fr_01,fr_015
0,Eine bahnbrechende Leistung,Une révolution de toute beauté,Eine ahnbrechende Leistung,Eine bahnurecjende Leistung,Eqine bhahnbrpechende Loeistung,Une révouution de toute beauté,Une régllution deftoute beauté,Une rrévkolwution de toutecbeauté
1,Ein Beispiel dafür ist die Entwicklung von neu...,C'est le cas de la définition de nouveaux conc...,Ein Beispiel dafü ist die Eyntwicklung von neu...,Ein Beipiel dafür isgt die Entwicgklung von he...,EgnBeispiel dafür ist die Entrwycklung vonieue...,C'est le cas de la définition de nouveaux oonc...,C'est kce cas de la définationb de lnouveax co...,C'est hle caf de la défyiition de nouveaux nca...
2,"""Eine wesentliche Motivation für diese Arbeit ...",Une motivation importante pour ces travaux éta...,Eine wesentliche Motivation für dieve Arbeitr ...,"""Eie wesentliche Motrvain für diese Arbeit yar...","a""Ena wesentlxche otivation für viese Arbeeit ...",Une motivation importante pour ces travaux éta...,Unwe motivation mnportantve pour ces travau nt...,Unmotisvatbon importante piau estravaux était ...
3,Gleichzeitig muss der Feuerwehrmann volles Ver...,"De son côté, le pompier doit avoir une confian...",Gleichzeitig mussder Feuerwihrmann yolles Veqr...,kleichzeitig muss der Feuerawehrmann olles Ver...,leicyzeiig mios der Feuerweqjhrmann volles Ver...,"De som côté, le pompier doit bvoir une confian...","De son côté,le pmpiper doitaoir une confrance ...","De sonb côtzé, le pompierv doot avoifrw une co..."
4,Diese begann im Mai 2015 und wird einen Schwer...,Cette dernière a été lancée en mai 2015 et se ...,Diese begann imf Mai 2015 und wrd einen Sfchwe...,Diese begsnn im Mai 201 unnd wird einen Schwer...,Diese byganhm Mai 2015 und wird hinen Sqfchwer...,Ctte dernière a été laiqncée en mai 2015 et se...,Cette dernièrde a étél lanwé en mae 2015 et se...,Cettey dnrnxière a été lancée ern mtai2015 ets...
5,Diese Steuerung ermöglicht den kabellosen Betr...,Cette sorte de contrôle tient compte de l'acti...,Diese Steuirung ermöglicht den kabehlosen Betr...,Diese steuerung eemögichtxdesn dkabellsen Betr...,iesl Steurerung rmögicht den kanbeiloren Bemtr...,Cette sorte de contrôle tient compte de l'acti...,Cette sorte de contôle tient compvte de l'acti...,oett sorge dewcontrle tient compte de h'acqion...
6,Doch diese Einschätzung steht im krassen Gegen...,Ceci n'est pas cohérent avec les possibilités ...,Doch diese Einschätzung steht im krassen Gegen...,Dkoch dise Einschätzung stevt im krassen Gegen...,Doch diese Einnfchäutzung steht izm ukrchssen ...,Ceci n'est pas cohérent avec les wossibilités ...,Ceci 'est pasfcohérent avec lles posyibilités ...,Ceci n'est pas coéroent tdvec le possgibilits ...
7,"""Diese Art von Energiequellenkonzeption hat ei...",«Ce genre de concept de sources d'énergie a un...,"""Diese Art hvon Energiequellemknzeption hat ei...","""Diese Art vopn Energiequelflznkonzeptionhat e...","""Diese Artsvon Energiequellenkonzepion hat eun...",«Ce genre de concept de sources d'énergie a un...,«Cwe genre de concefpt de sources d'énerieeaun...,«de genre de ocept de sources y'énergse a unim...
8,"Diese ""Art grüner Klebstoff"" (wie die Forscher...",Cette «sorte de substance visqueuse verte» (qu...,"Diese ""Art grküner Klebstof"" (wje die Forscher...","Diese ""Axrt grüznlr Klebstoff"" (wie dhe xForsc...","Dfstese ""At grüner lebstdtff"" (wie die Forschv...",Cette «sorte de srubstuance visqueuse verte» (...,etje «sorte mde substancn visqueuse verxte» (q...,Cetlte «sorte de substannce visqucus vere» (qu...
9,Diese Form der High-Impact-Physik scheint zwar...,Ce type de physique à fort impact peut sembler...,Diese Form der High-Impact-Physik scheint zwar...,Diese Form yer Highc-ompact-Physik skcheint zw...,Diese For de High-dmpact-Physik ssqheint zwar ...,Cewtype de physique à frt impact peut sembler ...,Ce tpe de physique fbrt impact put sekbler él...,Cev tupede plhysiquer à forimpact peuyt semdbl...


In [ ]:
mono_df.to_csv("/content/drive/My Drive/UZH/MA/Saved Dataframes/mono_df.csv")
cross_df.to_csv("/content/drive/My Drive/UZH/MA/Saved Dataframes/cross_df.csv")

In [ ]:
mono_df = pd.read_csv("/content/drive/My Drive/UZH/MA/Saved Dataframes/mono_df.csv")
cross_df = pd.read_csv("/content/drive/My Drive/UZH/MA/Saved Dataframes/cross_df.csv")

In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

# Ensure you have the necessary NLTK data
nltk.download('punkt')

# Example DataFrame

def calculate_avg_lengths(df, text_column):
    sentence_lengths = []
    token_lengths = []

    for text in df[text_column]:
        sentences = sent_tokenize(text)
        sentence_lengths.extend([len(sentence) for sentence in sentences])
        token_lengths.extend([len(word_tokenize(sentence)) for sentence in sentences])

    avg_sentence_length = sum(sentence_lengths) / len(sentence_lengths)
    avg_token_length = sum(token_lengths) / len(token_lengths)

    return avg_sentence_length, avg_token_length

avg_sentence_length, avg_token_length = calculate_avg_lengths(mono_df, 'de')
print(f'Average Sentence Length: {avg_sentence_length}')
print(f'Average Token Length: {avg_token_length}')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Average Sentence Length: 85.93580755099033
Average Token Length: 15.931082834639682


In [ ]:
avg_sentence_length, avg_token_length = calculate_avg_lengths(mono_df, 'fr')
print(f'Average Sentence Length: {avg_sentence_length}')
print(f'Average Token Length: {avg_token_length}')

Average Sentence Length: 91.5571472230442
Average Token Length: 17.951960532251885


In [ ]:
avg_sentence_length, avg_token_length = calculate_avg_lengths(cross_df, 'de')
print(f'Average Sentence Length: {avg_sentence_length}')
print(f'Average Token Length: {avg_token_length}')

Average Sentence Length: 155.26126632037062
Average Token Length: 23.16201038888109


In [ ]:
avg_sentence_length, avg_token_length = calculate_avg_lengths(cross_df, 'fr')
print(f'Average Sentence Length: {avg_sentence_length}')
print(f'Average Token Length: {avg_token_length}')

Average Sentence Length: 163.06176525288515
Average Token Length: 27.425769129119722


# Augmenting English Parallel Datasets


In [ ]:
import pandas as pd
import xml.etree.ElementTree as ET
import os

# Path to folder
path_to_datasets = "/content/drive/My Drive/UZH/MA/Datasets/"

# Function to parse TMX files and convert them into DataFrames
def parse_tmx(file_path):
    if not os.path.isfile(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")

    tree = ET.parse(file_path)
    root = tree.getroot()
    namespaces = {'xml': 'http://www.w3.org/XML/1998/namespace'}

    translations = []

    for tu in root.findall('.//tu'):
        de_text = None
        fr_text = None
        en_text = None

        for tuv in tu.findall('tuv'):
            lang = tuv.attrib['{http://www.w3.org/XML/1998/namespace}lang']
            seg = tuv.find('seg').text
            if lang == 'de':
                de_text = seg
            elif lang == 'fr':
                fr_text = seg
            elif lang == 'en':
                en_text = seg

        if de_text and en_text:
            translations.append({'de': de_text, 'en': en_text})
        if fr_text and en_text:
            translations.append({'fr': fr_text, 'en': en_text})

    return pd.DataFrame(translations)

# Full paths to TMX files
cross_en_fr_file_path = os.path.join(path_to_datasets, 'ELRC-CORDIS_News_de-fr', 'en-fr.tmx')
cross_en_de_file_path = os.path.join(path_to_datasets, 'ELRC-CORDIS_News_de-fr', 'de-en.tmx')

# Load datasets
cross_en_fr = parse_tmx(cross_en_fr_file_path)
cross_en_de = parse_tmx(cross_en_de_file_path)

# Display the dataframes
print(cross_en_fr.head())
print(cross_en_de.head())


                                                  fr  \
0  La "pourriture blanche" ou sclérotiniose est u...   
1  les actions de type A couvrent les projets de ...   
2                         a) Aéronautique et espace    
3  3 A.1 Conception des aéronefs et intégration d...   
4                     3 A.2 Production des aéronefs    

                                                  en  
0  `White mould' or `Sclerotinia rot' is a very d...  
1  A actions cover demonstration projects impleme...  
2                          a) Aeronautics and space   
3    3 A.1 Aircraft design and systems integration;   
4                        3 A.2 Aircraft production;   
                                                  de  \
0  A. africanus lebte vor über zwei Millionen Jah...   
1  A. Aktionen auf Kostenteilungsbasis, die aus Z...   
2  A. Aktionen auf Kostenteilungsbasis und konzer...   
3  A. Aktionen in Bezug auf Demonstrationsprojekt...   
4  A. Aktionen mit Kostenteilung unter Bezuschuss... 

In [ ]:
import pandas as pd
import random

# Function to introduce random noise
def introduce_ocr_like_noise(text, noise_rate=0.05):
    characters = list(text)
    n = len(characters)
    num_changes = int(n * noise_rate)
    for _ in range(num_changes):
        operation = random.choice(["delete", "insert", "substitute"])
        index = random.randint(0, n - 1)

        if operation == "delete":
            characters.pop(index)
        elif operation == "insert":
            characters.insert(index, random.choice("abcdefghijklmnopqrstuvwxyz"))
        elif operation == "substitute":
            characters[index] = random.choice("abcdefghijklmnopqrstuvwxyz")

        # adjust length for next iteration
        n = len(characters)

    return ''.join(characters)

# Function to add noisy columns to the dataframe
def add_noisy_columns(df, lang, noise_levels):
    for noise in noise_levels:
        new_column_name = f"{lang}_{str(noise).replace('.', '')}"
        df[new_column_name] = df[lang].apply(introduce_ocr_like_noise, noise_rate=noise)
    return df

# Define noise levels
noise_levels = [0.05, 0.1, 0.15]

# Add noisy columns to cross_en_fr
cross_en_fr = add_noisy_columns(cross_en_fr, 'en', noise_levels)
cross_en_fr = add_noisy_columns(cross_en_fr, 'fr', noise_levels)

# Add noisy columns to cross_en_de
cross_en_de = add_noisy_columns(cross_en_de, 'de', noise_levels)
cross_en_de = add_noisy_columns(cross_en_de, 'en', noise_levels)

# Display the modified dataframes
print(cross_en_fr.head())
print(cross_en_de.head())


                                                  fr  \
0  La "pourriture blanche" ou sclérotiniose est u...   
1  les actions de type A couvrent les projets de ...   
2                         a) Aéronautique et espace    
3  3 A.1 Conception des aéronefs et intégration d...   
4                     3 A.2 Production des aéronefs    

                                                  en  \
0  `White mould' or `Sclerotinia rot' is a very d...   
1  A actions cover demonstration projects impleme...   
2                          a) Aeronautics and space    
3    3 A.1 Aircraft design and systems integration;    
4                        3 A.2 Aircraft production;    

                                              en_005  \
0  `White mouzld' or vSclerotinia rot'pis a zery ...   
1  A action cover demonstration projects implemen...   
2                           a) Aeronauics and space    
3     3 A.1 Aircrat design and systems integration;    
4                        3 A.2 Aircraftwproduc

In [ ]:
# Set the maximum number of columns displayed
pd.set_option('display.max_columns', None)  # None = unlimited

#display all column
display(cross_en_fr.head(10))

,fr,en,en_005,en_01,en_015,fr_005,fr_01,fr_015
0,"La ""pourriture blanche"" ou sclérotiniose est u...",`White mould' or `Sclerotinia rot' is a very d...,`White mouzld' or vSclerotinia rot'pis a zery ...,Whitp mouid' or `Sclerotinia ros' is a venry d...,`Whitelmould' orvl`Sclerotnira rot' s a very l...,"La ""pourriture blanche"" ou sclérotinjose est u...","La ""pourritr blanche"" ou sclérkotiniose est un...","La ""pourriture blanche"" ou sclérotiriose est u..."
1,les actions de type A couvrent les projets de ...,A actions cover demonstration projects impleme...,A action cover demonstration projects implemen...,A actionsvcovqer demonstration projects mpleme...,A acbtions cover demonstrravion proaects impwe...,les actions de type A ouvrent les projets e dé...,les actions de type A couvrent les projetsp de...,ls acqtions detpe Aixouvrenst les pojets de dé...
2,a) Aéronautique et espace,a) Aeronautics and space,a) Aeronauics and space,a) Aeyonautics mand space,aa) Aeronautims and sace,a) Aérnautique et espace,a) Aérionnautique et espace,a) Aéroautiue met espace
3,3 A.1 Conception des aéronefs et intégration d...,3 A.1 Aircraft design and systems integration;,3 A.1 Aircrat design and systems integration;,A.1 Aircrwft design and systems pntegrationl,r A.1 Arcrafz desitn azd systems itegration;,3 A.1 Concetion des aéronefs en intégration de...,3 A.1 Concception des aéroneqfs ea intégraion ...,3o A.1 Conception des aérnefv et intékration d...
4,3 A.2 Production des aéronefs,3 A.2 Aircraft production;,3 A.2 Aircraftwproduction;,3 A.2 Aircraft produgtiokn;,3 Ak.2 Aurcraft produtmon;,3 A.2 Production desaéronefs,3 A.2 roductiwon des aérodnefs,3 rA.2 Productione deos aséronefs
5,une documentation d'appui.,A background document.,pA background document.,A baground document.,A backgrzund dcumenta,une documentation d'appui.,une documentation 'apui.,une documsentoion d'appui.
6,Une infection bactérienne pouvant causer des h...,A bacterial infection which can cause disabili...,A bacterial irnfectfion which can caxuse disab...,A beacteral nfection whic can cause disabzlity...,A bacerial irfiekction which can cause dyesabi...,Une infectin bactérienne pouvant qauser des ha...,Une infectio lbactérienne pouvant causer des h...,Une invuetion batdripenn ouvan gaser desmfandi...
7,Un aéroport sans bagages à l'horizon?,A baggage-free airport on the horizon?,A baggage-free airporet on the horizon?,Abaage-free airport on the horizon?,A baggage-freeyairpor fnthe horzon?,Un aéroport sans bagages à l'horizon?,Un aroport hsans bagages àl'horizon?,Un aéroport saasbagags àl'horizovn?
8,Une étude de marché des boulangeries a égaleme...,A bakery market study was also prepared and ha...,A bakey market study was also prepared and has...,A bakery mrket studywas also prepared kand hs ...,bakery market etuody waq also aphrepared anjd...,Une étudet de marché des boulangeries a égalem...,Uxe étuae de mmarché des boulangeries a égalem...,Une étude de marché nes kboulangeriesa ggaleme...
9,une campagne de mesures par ballons lancée de ...,A balloon measurement campaign was launched fr...,A bdlloon measurement campaint was launched fr...,A balloon measurementcampaign was daunuced frc...,o alloone meavurement cmaign was lunched from ...,ue campagne de mesures par ballos lancéepde Ki...,uine campagne hde mesurems par ballons lancée ...,une calxpagnede mesunres pwr ballwzns lancée d...


In [ ]:
# Set the maximum number of columns displayed
pd.set_option('display.max_columns', None)  # None = unlimited

#display all column
display(cross_en_de.head(10))

,de,en,de_005,de_01,de_015,en_005,en_01,en_015
0,A. africanus lebte vor über zwei Millionen Jah...,A. africanus lived in southern Africa over two...,A. africanus lebte vorlüber zwe Millionen Jahe...,A. jfricanus lbe vor über zwei Millionen Jarn ...,A. afrlicanus lebte lvor über zwei Mnlionezn J...,A. africanus lived in southern Africaover two ...,A. afwricanus lived inysoutern Africa ovprtwo ...,A. lfricanus lived in southen Afrca over twox ...
1,"A. Aktionen auf Kostenteilungsbasis, die aus Z...",A. Shared cost actions resulting from preparat...,"A. Aktionen auf Kostenteilungsbasis, ie ausZus...","A. Aktionen auf Kosenzteiungwbasts, die aus Zu...","A. Aktbnjn aufk Kostenteiglungsbasis,dixe aus ...",A. Sharedcost actions resulting from preparatt...,. Sharrd cost actionns resultinx from preparkt...,A Sared cost actionxs resulting fhrom praepara...
2,A. Aktionen auf Kostenteilungsbasis und konzer...,A. Shared cost actions and concerted actions r...,A. Aktionen af Kostenteilungsasis nd konertert...,A. Aktioinen auf Kosdtenoeilungsbjasis und khn...,A. Aktioen auf Kostenteilungsbasispund konzert...,A. Shared cost actions and concerted actions r...,A. Shared cost actiins and concerted arctnons ...,A. Share cost tctins andconcerted actons desuh...
3,A. Aktionen in Bezug auf Demonstrationsprojekt...,A actions cover demonstration projects impleme...,A. Aktionen in Bezug auf Demenstrationsprojekj...,A. Aktionrn in Bezug auf Demonnstrationsprojek...,A. nAktionen in Bezug agmf Demonstrationsproje...,A actwons cover demonstration projects impleme...,Aqarctiogns cover demonstratio pojects impleme...,A atiks coverdemnastration prorcts implementin...
4,A. Aktionen mit Kostenteilung unter Bezuschuss...,A. Shared-cost actions resulting from preparat...,A. Aktionen miatKosteteilung unter Beszuschuss...,A. Aktionen mit qostentellungunter Bezuschussu...,A.rAktionen miyt Kostenbeiluncg unterpBezuchus...,A. Shaped-cost actions rsulting from preparato...,A. Shared-cost actons resultcingx yfroe prepar...,A. Sxared-cost actios rwesulting from preparat...
5,a) Aktuelle technologische Trends in diesem Be...,"a) technological current trends in the area, w...",a) Aktuelle tehnologische Trends in diesem Ber...,a) Altunaelle thchnologishe Trends min udiesem...,a) Aktuelole schnolohichelTrends in xdiesem Br...,"a) technxlogical current treds n the area, wha...","a)c tehnological current trends in the area, w...","a)ttecphnologhcalcurrtent tbrendsj in th ara, ..."
6,A. Ausbildungsstipendien:,A. Grants for training:,A. Ausbildungsstipenien:,Ai. Ausbildungsstipendien:g,A.Ausbildungosstnpendien:,A. Grats for training:,A. Grantsi fok training:,A. Grants wor tracning:
7,(a): Biowissenschaften und -technologien:,(a): Life Sciences and Technologies:,(a): Biowssenxschaften und -technologien:,(a): Biowisssenschaddften und -technololien:,(a) xBiowissenschaftn uni -tchnologien:,(a): Life ciences and Technologies:,(p): Lipe Sciences andmTechnologies:,():jife Sciencesv and Technologes:
8,A - Brennstoffzellen und H2: Brennstoffzellen ...,A - fuel cells and H2: application driven fuel...,A - Brennstoffzellen und H2: Brennstoffzellen ...,A - Brennsoffjzelren und Hv2: Brennstoffzellen...,o - Brenntoffzellegn und H2: Brenntffelsen für...,A - fuel cellcs and H2: application driven fue...,A - fuel cells and H2 applicatiokn drivet fune...,r - fuel cells an H2: anplicaixn drive fuel ce...
9,"""A-CUTE-TOX"" ist ein umfangreiches integrierte...",A-CUTE-TOX is a large-scale Integrated Project...,"""A-CUTE-TOX"" ist ein umfangriches integriertex...","""h-CUeTEy-TOXa ist ein umfangreiches integtier...","""ACUTETO""r ist ein wumfiangreiches inhtegriser...",A-CUTE-TOX isy large-scale Integratedh Project...,"A-CUTE-TOX is a lrge-sbale Integrated Project,...","A-CUTE-TOX is a lbre-scale unterted Projpct, t..."


In [ ]:
cross_en_fr.to_csv("/content/drive/My Drive/UZH/MA/Saved Dataframes/cross_en_fr.csv")
cross_en_de.to_csv("/content/drive/My Drive/UZH/MA/Saved Dataframes/cross_en_de.csv")